In [480]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

In [1059]:
df_train=pd.read_csv('train_data/train.csv')
df_cust=pd.read_csv('train_data/customer_demographics.csv')
df_txn=pd.read_csv('train_data/customer_transaction_data.csv')
df_campaign=pd.read_csv('train_data/campaign_data.csv')
df_coupon_item_map=pd.read_csv('train_data/coupon_item_mapping.csv')
df_item_data=pd.read_csv('train_data/item_data.csv')
df_test=pd.read_csv("test_data.csv")
#need to merge train & test set

In [1060]:
df=pd.concat([df_train, df_test],axis=0,ignore_index=True,verify_integrity=True)  # verify_integrity - prevents duplicates

In [1061]:
#Target column
print(df['redemption_status'].value_counts())
print('NOT null values : ',df['redemption_status'].notnull().sum())
print('null values : ',df['redemption_status'].isnull().sum())
print(df.shape)

0.0    77640
1.0      729
Name: redemption_status, dtype: int64
NOT null values :  78369
null values :  50226
(128595, 5)


# Need to join column values from all the csv files
df_train.join(df_cust,on='customer_id',lsuffix='_train') # preserves index but many values null

left vs inner join: df1.join(df2) does a left join by default (keeps all rows of df1), 
but df.merge does an inner join by default (returns only matching rows of df1 and df2).
Hence we will take df2

In [1058]:
df_cust.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [998]:
df=pd.merge(df,df_cust,on='customer_id',how='outer')

In [1062]:
df=df.merge(df_campaign,on='campaign_id',how='outer')
# is same as pd.DataFrame(df,df_campaign,on='campaign_id')

In [1063]:
print(df.shape)
print(df.id.count())
print('target null values : ',df['redemption_status'].isnull().sum())

(128595, 8)
128595
target null values :  50226


In [1064]:
df_txn.columns

Index(['date', 'customer_id', 'item_id', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount'],
      dtype='object')

In [1065]:
# before we include df_txn , it needs work
df_txn.drop(['selling_price','other_discount','quantity'],inplace=True,axis=1)
df_txn.drop_duplicates(keep='first', inplace=True)  # before (1324566, 4) after (1320441, 4)
print(df_txn.shape)

(1320441, 4)


In [1003]:
#one coupon applicable on multiple items, we cannot directly merge mapping df to our df

In [1066]:
# TAKES TIME
df=df.merge(df_coupon_item_map,on='coupon_id', how='inner')
df=df.merge(df_item_data,on='item_id',how='inner')
df_combo=df.merge(df_txn,on=['item_id','customer_id'],how='inner').drop_duplicates()
print(df_combo.shape) #(183954, 20)

(263914, 14)


In [1067]:
df_combo.id.value_counts()

96691     776
22510     668
27110     574
112119    572
29656     524
61066     480
60475     478
118668    468
27690     468
94836     465
24701     465
87988     457
7139      457
2592      425
39886     401
63738     399
93701     398
65138     382
69141     369
59972     367
52594     365
73452     354
55721     351
12290     349
10614     349
31328     346
95304     346
7361      343
88130     342
64206     341
         ... 
126911      1
112564      1
13957       1
104113      1
76705       1
3970        1
39831       1
37782       1
900         1
117437      1
25224       1
14979       1
12930       1
102838      1
76193       1
59801       1
18829       1
31115       1
102582      1
106672      1
84141       1
49308       1
47251       1
41104       1
26761       1
106416      1
98220       1
71585       1
28558       1
2049        1
Name: id, Length: 26801, dtype: int64

In [1068]:
df_combo.isnull().sum()
# Count non-NA cells for each column or row.
#print(df_combo['marital_status'].count())  # Null-71538     not_null-112522  TOT-184060   = 38% are null
#63.40% null for no_of_children column

campaign_id               0
coupon_id                 0
customer_id               0
id                        0
redemption_status    124147
campaign_type             0
start_date                0
end_date                  0
item_id                   0
brand                     0
brand_type                0
category                  0
date                      0
coupon_discount           0
dtype: int64

In [1008]:
df_combo.dropna(subset=['family_size'],inplace=True)

In [1010]:
df_combo['child_count']=df_combo['no_of_children']  #creating new column

In [1011]:
df_combo.loc[(df_combo['family_size'].str.contains('1')),'child_count']=0    # age can also be included if req
df_combo.drop('no_of_children',axis=1,inplace=True)

# other way to do this
#df_combo['child_count'] = [0 if b=='1' else 0 for b in df_combo['family_size']]

In [1012]:
#Assumption : if child count is non-zero , marital status should be married.[nan, 'Married', 'Single']
df_combo['child_count'].replace('3+','3 ',inplace=True)
df_combo['child_count'] = pd.to_numeric(df_combo['child_count']) # column type itself changed
df_combo['child_count'] = df_combo['child_count'].astype(float)  # cant be converted to int since nan values present
print(df_combo['child_count'].unique())

[ 1.  0.  3. nan  2.]


In [1013]:
df_combo[(df_combo['child_count']>0) & ( df_combo['family_size']==1 ) ]
#Insight : So it is safe to assume marital_status is married if child_count>0

,campaign_id,coupon_id,customer_id,id,redemption_status,age_range,marital_status,rented,family_size,income_bracket,campaign_type,start_date,end_date,item_id,brand,brand_type,category,date,coupon_discount,child_count


In [1014]:
df_combo['family_size'].replace('5+','5',inplace=True)
df_combo['family_size'] = pd.to_numeric(df_combo['family_size'])
print(df_combo['family_size'].unique())

[3 1 5 2 4]


In [ ]:
df_combo['marital']=df_combo['marital_status']
df_combo.loc[(df_combo['child_count']>0),'marital']='Married'
df_combo.drop('marital_status',axis=1,inplace=True)

In [1016]:
df_combo['marital'].isnull().sum()  # 71538 -> 56044 null values reduced

56010

In [1017]:
# Assuming when family_size=1 & child_count=0 THEN marital=Single
df_combo.loc[(df_combo['family_size']==1) & (df_combo['child_count']<1),'marital']='Single'
print(df_combo['marital'].isnull().sum())

16934


In [1018]:
# Assuming when family_size=2 & marital=Married THEN child_count=0   So,42701 rows can be filled

#        df_combo[(df_combo['family_size']==2) & (df_combo['marital']=='Married') & (df_combo['child_count'].isnull() )]

In [1019]:
df_combo.loc[(df_combo['family_size']==2) & (df_combo['marital']=='Married'),'child_count']=0
print(df_combo['child_count'].isnull().sum())

20328


In [ ]:
df_combo['age_range'].value_counts()

In [1021]:
age_dict={
    "18-25":22,
    "26-35":31,
    "36-45":41,
    "46-55":51,   
    "56-70":63,
    "70+":71
}

In [1022]:
df_combo['Age']=df_combo['age_range'].map(age_dict)
df_combo.drop('age_range',inplace=True,axis=1)

brand_type=> Local/Established,
category=> 16 values

In [1071]:
#one hot encoding for brand_type column
df_combo.columns

Index(['campaign_id', 'coupon_id', 'customer_id', 'id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date', 'item_id', 'brand',
       'brand_type', 'category', 'date', 'coupon_discount'],
      dtype='object')

In [1072]:
df_combo['brand_type'].value_counts()

Established    209328
Local           54586
Name: brand_type, dtype: int64

In [1073]:
df1=pd.get_dummies(df_combo['brand_type'],drop_first=True)
df_combo.drop(['brand_type'],axis=1,inplace=True)
df_combo=pd.concat([df_combo,df1],axis=1)

In [1074]:
df_combo['Local'].value_counts()

0    209328
1     54586
Name: Local, dtype: int64

In [1027]:
# one hot encoding using LabelEncoder first    FOR column - category
# LablelEncoder limitation -> it may introduce sequence relation betn rows like 1,2,3,.. which is not there
# Hence OneHotEncoder is must after that

In [1028]:
#Another way to get categorical columns
df_combo.select_dtypes(include=[object]).columns

#OR

#cat_col=list(df_combo.dtypes[df_combo.dtypes=="object"].index)

Index(['campaign_type', 'start_date', 'end_date', 'category', 'date',
       'marital'],
      dtype='object')

In [1075]:
df2=pd.get_dummies(df_combo['category'],drop_first=True,prefix='cat')
df_combo.drop(['category'],axis=1,inplace=True)
df_combo=pd.concat([df_combo,df2],axis=1)

In [1076]:
df_combo.columns = df_combo.columns.str.strip()
df_combo.columns = df_combo.columns.str.replace('(', '')
df_combo.columns = df_combo.columns.str.replace(')', '')
df_combo.columns = df_combo.columns.str.replace(',', '')
df_combo.columns = df_combo.columns.str.replace(' ', '')
# str.lower() can also be used

In [1077]:
df3=pd.get_dummies(df_combo['campaign_type'],drop_first=True,prefix='camp_type')
df_combo.drop(['campaign_type'],axis=1,inplace=True)
df_combo=pd.concat([df_combo,df3],axis=1)

In [1032]:
#Converting dtype for marital & one hot encoding
print(df_combo['marital'].dtype)
print(type(df_combo['marital'][0]))
print(df_combo['marital'].unique())   # TOT 167119 (married + single)

object
<class 'str'>
['Married' 'Single' nan]


In [1033]:
df4=pd.get_dummies(df_combo['marital'],drop_first=True,prefix='marital')
df_combo.drop(['marital'],axis=1,inplace=True)
df_combo=pd.concat([df_combo,df4],axis=1)

In [1034]:
df_combo.head(0)  # added new column marital_Single

,campaign_id,coupon_id,customer_id,id,redemption_status,rented,family_size,income_bracket,start_date,end_date,...,cat_PackagedMeat,cat_Pharmaceutical,cat_PreparedFood,cat_Salads,cat_Seafood,cat_Skin&HairCare,cat_Travel,cat_Vegetablescut,camp_type_Y,marital_Single


In [1098]:
#BACKUP=df_combo.copy()            ###### BACKUP #########
#df_combo=BACKUP

#good pracitce to save this data to csv file
#          BACKUP.to_csv("BACKUP.csv")

In [1099]:
df_combo['redemption_status'].value_counts()   #93955 not null  #90105 NULL  TOT 184060   48% null values

0.0    121223
1.0     18544
Name: redemption_status, dtype: int64

In [1100]:
df_combo.drop(['start_date', 'end_date', 'date'],axis=1,inplace=True)

In [1101]:
train=df_combo[df_combo['redemption_status'].notnull()]
test=df_combo[df_combo['redemption_status'].isnull()]

In [1102]:
print(train.id.count())
print(test.id.count())   #we need 50226

139767
124147


In [1103]:
test.drop(['redemption_status'],axis=1,inplace=True)
print(test.shape)
print(test.columns)
#we need to make test same as df_test for the original columns
#['id', 'campaign_id', 'coupon_id', 'customer_id']

(124147, 24)
Index(['campaign_id', 'coupon_id', 'customer_id', 'id', 'item_id', 'brand',
       'coupon_discount', 'Local', 'cat_DairyJuices&Snacks',
       'cat_Flowers&Plants', 'cat_Garden', 'cat_Grocery', 'cat_Meat',
       'cat_Miscellaneous', 'cat_NaturalProducts', 'cat_PackagedMeat',
       'cat_Pharmaceutical', 'cat_PreparedFood', 'cat_Salads', 'cat_Seafood',
       'cat_Skin&HairCare', 'cat_Travel', 'cat_Vegetablescut', 'camp_type_Y'],
      dtype='object')


In [1104]:
test.drop_duplicates(inplace=True)
#test.drop_duplicates(subset='id', keep="first",inplace=True)
test.shape

(66574, 24)

In [1084]:
test['id'].count()
# Length: 50226
# i.e. if take unique values of 'id' ,then we have only 50226 records as expected

124147

In [1105]:
test_cat=test.merge(df_test,on=['id','campaign_id','coupon_id','customer_id'],how='outer')

In [1086]:
test_cat[test_cat['id']==3].iloc[:15,7:14]

,brand,date,coupon_discount,Local,cat_DairyJuices&Snacks,cat_Flowers&Plants,cat_Garden
78850,1075.0,2012-08-24,-35.62,0.0,0.0,0.0,0.0
78851,1075.0,2013-05-02,0.00,0.0,0.0,0.0,0.0
78852,1075.0,2012-08-31,0.00,0.0,0.0,0.0,0.0
78853,1075.0,2012-10-26,0.00,0.0,0.0,0.0,0.0
78854,1075.0,2012-11-30,0.00,0.0,0.0,0.0,0.0
78855,1075.0,2013-04-19,0.00,0.0,0.0,0.0,0.0
78856,1075.0,2012-10-04,0.00,0.0,0.0,0.0,0.0
78857,1075.0,2012-11-23,0.00,0.0,0.0,0.0,0.0
78858,1075.0,2012-05-24,0.00,0.0,0.0,0.0,0.0
78859,1075.0,2012-04-25,0.00,0.0,0.0,0.0,0.0


In [1107]:
test_cat.drop_duplicates(subset='id', keep="first",inplace=True)
test_cat.id.value_counts()

26149     1
46428     1
34138     1
40281     1
103768    1
46391     1
64853     1
64651     1
118492    1
77135     1
81229     1
13644     1
3403      1
1354      1
115707    1
5448      1
88389     1
95556     1
114217    1
83266     1
23873     1
107838    1
113981    1
36187     1
48524     1
35835     1
107870    1
40313     1
126327    1
58742     1
         ..
35428     1
111203    1
47714     1
41569     1
23134     1
90745     1
32641     1
94843     1
60040     1
74385     1
10896     1
94814     1
55950     1
125027    1
117388    1
127627    1
67657     1
57993     1
84586     1
19068     1
39558     1
98949     1
111235    1
113282    1
108488    1
43648     1
21119     1
23166     1
17021     1
104087    1
Name: id, Length: 50226, dtype: int64

In [1108]:
#X_train=train.drop(['redemption_status','start_date', 'end_date', 'date'],axis=1)
X_train=train.drop(['redemption_status'],axis=1)
y_train=train['redemption_status']

In [1119]:
# delete this
#internal testing accuracy
A=train.drop(['redemption_status'],axis=1)
B=train['redemption_status']

X_train, X_test, y_train, y_test = train_test_split(A, B, test_size=0.30, random_state=42)

In [1112]:
import xgboost as xgb
classifier = xgb.XGBClassifier()

In [1120]:
#DataFrame.dtypes for data must be int, float or bool.
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [1121]:
pred=classifier.predict(X_test)
# training on X features and testing on Y features in not allowed
pred.shape

(41931,)

In [1122]:
np.count_nonzero(pred)

1549

In [1123]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
# BUt we need to check error  with ROC curve

[[36243   126]
 [ 4139  1423]]
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.94     36369
         1.0       0.92      0.26      0.40      5562

   micro avg       0.90      0.90      0.90     41931
   macro avg       0.91      0.63      0.67     41931
weighted avg       0.90      0.90      0.87     41931



In [ ]:
#128594 records in sample submission , exact count 50226
#my pred count is  28187
df_sub=pd.read_csv('sample_submission.csv')

In [1117]:
from sklearn.metrics import roc_auc_score

In [1124]:
#https://elitedatascience.com/imbalanced-classes

# Predict class probabilities
prob_y_2 = classifier.predict_proba(X_test)
 
# Keep only the positive class
prob_y_2 = [p[1] for p in prob_y_2]

print( roc_auc_score(y_test, prob_y_2) )   #0.91 with all test columns

0.9103431061789655
